In [ ]:
# 1. Importación de Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier

# Configuración de visualización
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Descargas necesarias de NLTK
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
# 2. Carga de Datos
# Asegúrate de que la ruta al archivo es correcta
file_path = './Dataset/cyberbullying_tweets.csv'
try:
    df = pd.read_csv(file_path)
    print(f"Dataset cargado con éxito. Dimensiones: {df.shape}")
    display(df.head())
except FileNotFoundError:
    print(f"Error: No se encontró el archivo en {file_path}")

In [ ]:
# Análisis de la distribución de clases
plt.figure(figsize=(10, 5))
sns.countplot(x='cyberbullying_type', data=df, palette='viridis')
plt.title('Distribución de Tipos de Ciberbullying')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# 3. Preprocesamiento de Texto

stop_words = set(stopwords.words('english')) # Cambiar a 'spanish' si usas el dataset traducido
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # Convertir a minúsculas
    text = str(text).lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenización y eliminación de stopwords
    words = [word for word in text.split() if word not in stop_words]
    # Lematización
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

print("Limpiando textos...")
df['clean_text'] = df['tweet_text'].apply(clean_text)
print("Limpieza completada.")

In [ ]:
# División de datos (Train/Test)
X = df['clean_text']
y = df['cyberbullying_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Datos de entrenamiento: {X_train.shape}")
print(f"Datos de prueba: {X_test.shape}")

In [ ]:
# 4. Definición de Modelos (El Array de Algoritmos)

# Aquí definimos la lista de modelos que queremos probar.
# Usamos Pipelines para incluir la vectorización (TF-IDF) como parte del modelo.

models_to_evaluate = [
    {
        'name': 'Logistic Regression',
        'model': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000)),
            ('clf', LogisticRegression(max_iter=1000, n_jobs=-1))
        ])
    },
    {
        'name': 'Linear SVC',
        'model': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000)),
            ('clf', LinearSVC())
        ])
    },
    {
        'name': 'Multinomial Naive Bayes',
        'model': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000)),
            ('clf', MultinomialNB())
        ])
    },
    {
        'name': 'Random Forest',
        'model': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000)),
            ('clf', RandomForestClassifier(n_estimators=100, n_jobs=-1))
        ])
    },
    {
        'name': 'Decision Tree',
        'model': Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000)),
            ('clf', DecisionTreeClassifier())
        ])
    }
]

print(f"Modelos configurados para evaluación: {len(models_to_evaluate)}")

In [ ]:
# 5. Bucle de Entrenamiento y Evaluación

results = []

for entry in models_to_evaluate:
    model_name = entry['name']
    model = entry['model']
    
    print(f"\n--- Entrenando {model_name} ---")
    
    # Entrenamiento
    model.fit(X_train, y_train)
    
    # Predicción
    y_pred = model.predict(X_test)
    
    # Métricas
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score (Weighted): {f1:.4f}")
    
    # Guardar resultados
    results.append({
        'Model': model_name,
        'Accuracy': acc,
        'F1_Score': f1,
        'y_pred': y_pred # Guardamos predicciones para matrices de confusión si se necesita
    })

In [ ]:
# 6. Visualización Comparativa de Métricas

results_df = pd.DataFrame(results)

# Gráfico de Barras para Accuracy y F1 Score
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.barplot(x='Accuracy', y='Model', data=results_df, palette='viridis')
plt.title('Comparación de Accuracy por Modelo')
plt.xlim(0, 1.0)

plt.subplot(1, 2, 2)
sns.barplot(x='F1_Score', y='Model', data=results_df, palette='magma')
plt.title('Comparación de F1 Score (Weighted) por Modelo')
plt.xlim(0, 1.0)

plt.tight_layout()
plt.show()

In [ ]:
# 7. Matriz de Confusión del Mejor Modelo

# Identificar el mejor modelo basado en F1 Score
best_model_row = results_df.loc[results_df['F1_Score'].idxmax()]
best_model_name = best_model_row['Model']
best_y_pred = best_model_row['y_pred']

print(f"El mejor modelo fue: {best_model_name}")

# Generar Matriz de Confusión
cm = confusion_matrix(y_test, best_y_pred)
labels = sorted(y.unique())

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.title(f'Matriz de Confusión - {best_model_name}')
plt.xlabel('Predicción')
plt.ylabel('Realidad')
plt.show()

In [ ]:
# Reporte detallado del mejor modelo
print(f"Reporte de Clasificación para {best_model_name}:\n")
print(classification_report(y_test, best_y_pred))